### SageMaker Feature Store Notebook showing use of Time Travel

This notebook is part of an AWS blog that shows how to use "Time Travel" by leveraging SageMaker Feature Store. This particular notebook (#2) is used to generate aggregate data attributes (i.e. averages and sums) from the raw transaction data generated in previous notebook (#1). 

In [1]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import json
import sys

role = get_execution_role()
sm_client = boto3.Session().client(service_name='sagemaker')
smfs_runtime = boto3.Session().client(service_name='sagemaker-featurestore-runtime')

#### Start by Deleting Feature Groups that we will re-create

In [2]:
# Use SageMaker default bucket
BUCKET = sagemaker.Session().default_bucket()
BASE_PREFIX = "sagemaker-featurestore-demo"

OFFLINE_STORE_BASE_URI = f's3://{BUCKET}/{BASE_PREFIX}'
# Note that FeatureStore will append this to prefix -> "sagemaker/us-east-1/offline-store/"

print(OFFLINE_STORE_BASE_URI)

s3://sagemaker-us-east-1-572539092864/sagemaker-featurestore-demo


In [ ]:
try:
    sm_client.delete_feature_group(FeatureGroupName='cc-agg-batch-fg') 
    print('deleted batch fg')
except:
    pass

In [ ]:
try:
    sm_client.delete_feature_group(FeatureGroupName='cc-agg-fg') # use if needed to re-create
    print('deleted fg')
except:
    pass

#### Make sure those feature groups are not in the list

In [3]:
sm_client.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'transaction-feature-group-03-20-32-44',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:572539092864:feature-group/transaction-feature-group-03-20-32-44',
   'CreationTime': datetime.datetime(2021, 3, 3, 20, 36, 27, 700000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'pyspark-fg',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:572539092864:feature-group/pyspark-fg',
   'CreationTime': datetime.datetime(2021, 4, 20, 18, 54, 48, 557000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'orders-feature-group-21-14-45-34',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:572539092864:feature-group/orders-feature-group-21-14-45-34',
   'CreationTime': datetime.datetime(2021, 4, 21, 14, 47, 4, 578000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 

#### Recreate the Feature Groups using Schema definition files
Each feature group contains configuration parameters for Offline and Online stores. The feature group uses a schema definition file (JSON) that dictates the feature names and types. Below we display these local schema files.

#### Schema files on in the local 'schema' folder

In [ ]:
!pygmentize schema/cc-agg-fg-schema.json

In [ ]:
!pygmentize schema/cc-agg-batch-fg-schema.json

In [4]:
def schema_to_defs(filename):
    schema = json.loads(open(filename).read())
    
    feature_definitions = []
    
    for col in schema['Features']:
        feature = {'FeatureName': col['name']}
        if col['type'] == 'double':
            feature['FeatureType'] = 'Fractional'
        elif col['type'] == 'bigint':
            feature['FeatureType'] = 'Integral'
        else:
            feature['FeatureType'] = 'String'
        feature_definitions.append(feature)

    return feature_definitions, schema['record_identifier_feature_name'], schema['event_time_feature_name']

In [5]:
def create_feature_group_from_schema(filename, fg_name, role_arn=None, s3_uri=None):
    schema = json.loads(open(filename).read())
    
    feature_defs = []
    
    for col in schema['features']:
        feature = {'FeatureName': col['name']}
        if col['type'] == 'double':
            feature['FeatureType'] = 'Fractional'
        elif col['type'] == 'bigint':
            feature['FeatureType'] = 'Integral'
        else:
            feature['FeatureType'] = 'String'
        feature_defs.append(feature)

    record_identifier_name = schema['record_identifier_feature_name']
    event_time_name = schema['event_time_feature_name']

    if role_arn is None:
        role_arn = get_execution_role()

    if s3_uri is None:
        offline_config = {}
    else:
        print(f'Creating Offline Store at: {s3_uri}')
        offline_config = {'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': s3_uri}}}
        
    sm_client.create_feature_group(
        FeatureGroupName = fg_name,
        RecordIdentifierFeatureName = record_identifier_name,
        EventTimeFeatureName = event_time_name,
        FeatureDefinitions = feature_defs,
        Description = schema['description'],
        Tags = schema['tags'],
        OnlineStoreConfig = {'EnableOnlineStore': True},
        RoleArn = role_arn,
        **offline_config)

#### Create the new Feature Groups using the schema definition 

In [ ]:
create_feature_group_from_schema('schema/cc-agg-fg-schema.json', 'cc-agg-fg', 
                                 role_arn=role, s3_uri=OFFLINE_STORE_BASE_URI)

In [ ]:
create_feature_group_from_schema('schema/cc-agg-batch-fg-schema.json', 'cc-agg-batch-fg', 
                                 role_arn=role, s3_uri=OFFLINE_STORE_BASE_URI)

#### Make sure the new Feature Groups exist

In [6]:
sm_client.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'transaction-feature-group-03-20-32-44',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:572539092864:feature-group/transaction-feature-group-03-20-32-44',
   'CreationTime': datetime.datetime(2021, 3, 3, 20, 36, 27, 700000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'pyspark-fg',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:572539092864:feature-group/pyspark-fg',
   'CreationTime': datetime.datetime(2021, 4, 20, 18, 54, 48, 557000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'orders-feature-group-21-14-45-34',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:572539092864:feature-group/orders-feature-group-21-14-45-34',
   'CreationTime': datetime.datetime(2021, 4, 21, 14, 47, 4, 578000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 

#### Describe each feature group
Note that each feature group gets its own ARN, allowing you to manage IAM policies that control access to individual feature groups. The feature names and types are displayed, and the record identifier and event time features are called out specifically. Notice that there is only an `OnlineStoreConfig` and no `OfflineStoreConfig`, as we have decided not to replicate features offline for these groups.

In [ ]:
sm_client.describe_feature_group(FeatureGroupName='cc-agg-fg')

In [7]:
sm_client.describe_feature_group(FeatureGroupName='cc-agg-batch-fg')

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:572539092864:feature-group/cc-agg-batch-fg',
 'FeatureGroupName': 'cc-agg-batch-fg',
 'RecordIdentifierFeatureName': 'cc_num',
 'EventTimeFeatureName': 'trans_time',
 'FeatureDefinitions': [{'FeatureName': 'tid', 'FeatureType': 'String'},
  {'FeatureName': 'cc_num', 'FeatureType': 'Integral'},
  {'FeatureName': 'num_trans_last_1d', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_amt_last_1d', 'FeatureType': 'Fractional'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'trans_time', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2021, 4, 15, 12, 27, 20, 670000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-us-east-1-572539092864/sagemaker-featurestore-demo'},
  'DisableGlueTableCreation': False,
  'DataCatalogConfig': {'TableName': 'cc-agg-batch-fg-1618489640',
   'Catalog': 'AwsDataCatalog',
   '

# Batch Ingestion
**This section of the notebook aggregates raw features into new derived features that is used for Fraud Detection model training/inference.**

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Create PySpark Processing Script](#Create-PySpark-Processing-Script)
1. [Run SageMaker Processing Job](#Run-SageMaker-Processing-Job)
1. [Explore Aggregated Features](#Explore-Aggregated-Features)
1. [Validate Feature Group for Records](#Validate-Feature-Group-for-Records)

### Background

- This notebook takes raw credit card transactions data (csv) generated by 
[notebook 0](./0_prepare_transactions_dataset.ipynb) and aggregates the raw features to create new features (ratios) via <b>SageMaker Processing</b> PySpark Job. These aggregated features alongside the raw original features will be leveraged in the training phase of a Credit Card Fraud Detection model in the next step (see notebook [notebook 3](./3_train_and_deploy_model.ipynb)).

- As part of the Spark job, we also select the latest daily aggregated features - `num_trans_last_1d` and `avg_amt_last_1d` grouped by `cc_num` (credit card number) and populate these features into the <b>SageMaker Online Feature Store</b> as a feature group. This feature group (`cc-agg-batch-fg`) was created in notebook [notebook 1](./1_setup.ipynb).

- [Amazon SageMaker Processing](https://aws.amazon.com/about-aws/whats-new/2020/09/amazon-sagemaker-processing-now-supports-built-in-spark-containers-for-big-data-processing/) lets customers run analytics jobs for data engineering and model evaluation on Amazon SageMaker easily and at scale. It provides a fully managed Spark environment for data processing or feature engineering workloads.

<img src="./images/batch_ingestion.png" />

### Setup

#### Imports 

In [8]:
from sagemaker.spark.processing import PySparkProcessor
import pandas as pd
import numpy as np
import sagemaker
import logging
import random
import boto3
import os

In [9]:
print(f'Using SageMaker version: {sagemaker.__version__}')

Using SageMaker version: 2.29.1


#### Setup Logger

In [10]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

In [11]:
logger.info('[Batch Aggregation using SageMaker PySpark Processing Job]')

[Batch Aggregation using SageMaker PySpark Processing Job]


#### Essentials

In [12]:
# Setup S3 prefixes for Spark Job

INPUT_KEY_PREFIX = os.path.join(BASE_PREFIX, 'raw')
OUTPUT_KEY_PREFIX = os.path.join(BASE_PREFIX, 'aggregated')
LOCAL_DIR = './data'

print(INPUT_KEY_PREFIX)
print(OUTPUT_KEY_PREFIX)

sagemaker-featurestore-demo/raw
sagemaker-featurestore-demo/aggregated


### Create PySpark Script
This PySpark script does the following:

1. Aggregates raw features to derive new features (ratios).
2. Saves the aggregated features alongside the original raw features into a CSV file and writes it to S3 - will be used in the next step for model training.
3. Groups the aggregated features by credit card number and picks selected aggregated features to write to SageMaker Feature Store (Online). <br>
<b>Note: </b> The feature group was created in the previous notebook (`1_setup.ipynb`)

In [16]:
%%writefile batch_aggregation.py
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, TimestampType, LongType
from pyspark.sql.functions import desc, dense_rank
from pyspark.sql import SparkSession, DataFrame
from  argparse import Namespace, ArgumentParser
from pyspark.sql.window import Window
import argparse
import logging
import boto3
import time
import sys
import os

TOTAL_UNIQUE_USERS = 10000 # Was 1000 -pmh
FEATURE_GROUP = 'cc-agg-batch-fg'

logger = logging.getLogger('sagemaker')
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())


feature_store_client = boto3.client(service_name='sagemaker-featurestore-runtime')


def parse_args() -> Namespace:
    parser = ArgumentParser(description='Spark Job Input and Output Args')
    parser.add_argument('--s3_input_bucket', type=str, help='S3 Input Bucket')
    parser.add_argument('--s3_input_key_prefix', type=str, help='S3 Input Key Prefix')
    parser.add_argument('--s3_output_bucket', type=str, help='S3 Output Bucket')
    parser.add_argument('--s3_output_key_prefix', type=str, help='S3 Output Key Prefix')
    args = parser.parse_args()
    return args
    

def define_schema() -> StructType:
    # Added 'tid' -pmh
    schema = StructType([StructField('tid', StringType(), True),
                         StructField('event_time', TimestampType(), True),
                         StructField('cc_num', LongType(), True),
                         StructField('amount', DoubleType(), True),
                         StructField('fraud_label', StringType(), True)])
    return schema


def aggregate_features(args: Namespace, schema: StructType, spark: SparkSession) -> DataFrame:
    logger.info('[Read Raw Transactions Data as Spark DataFrame]')
    transactions_df = spark.read.csv(f's3a://{os.path.join(args.s3_input_bucket, args.s3_input_key_prefix)}', \
                                     header=False, \
                                     schema=schema)
    
    logger.info('[Aggregate Transactions to Derive New Features using Spark SQL]')
    
    ## Note: Window w2 was '24 HOURS', changed to '7 DAYS'
    # TODO: change the names of the "*_1d" attributes below to "*_1w"

    query = """
    SELECT *, \
           avg_amt_last_60m/avg_amt_last_1d AS amt_ratio1, \
           amount/avg_amt_last_1d AS amt_ratio2, \
           num_trans_last_60m/num_trans_last_1d AS count_ratio \
    FROM \
        ( \
        SELECT *, \
               COUNT(*) OVER w1 as num_trans_last_60m, \
               AVG(amount) OVER w1 as avg_amt_last_60m, \
               COUNT(*) OVER w2 as num_trans_last_1d, \
               AVG(amount) OVER w2 as avg_amt_last_1d \
        FROM transactions_df \
        WINDOW \
               w1 AS (PARTITION BY cc_num order by cast(event_time AS timestamp) RANGE INTERVAL 60 MINUTES PRECEDING), \
               w2 AS (PARTITION BY cc_num order by cast(event_time AS timestamp) RANGE INTERVAL 7 DAYS PRECEDING) \
        ) 
    """
    
    transactions_df.registerTempTable('transactions_df')
    aggregated_features = spark.sql(query)
    return aggregated_features


def write_to_s3(args: Namespace, aggregated_features: DataFrame) -> None:
    logger.info('[Write Aggregated Features to S3]')
    aggregated_features.coalesce(1) \
                       .write.format('com.databricks.spark.csv') \
                       .option('header', True) \
                       .mode('overwrite') \
                       .option('sep', ',') \
                       .save('s3a://' + os.path.join(args.s3_output_bucket, args.s3_output_key_prefix))
    
def group_by_card_number(aggregated_features: DataFrame) -> DataFrame: 
    logger.info('[Group Aggregated Features by Card Number]')
    window = Window.partitionBy('cc_num').orderBy(desc('event_time'))
    sorted_df = aggregated_features.withColumn('rank', dense_rank().over(window))
    grouped_df = sorted_df.filter(sorted_df.rank == 1).drop(sorted_df.rank)
    # Added 'tid' and 'event_time' features -pmh
    sliced_df = grouped_df.select('tid', 'cc_num', 'num_trans_last_1d', 'avg_amt_last_1d', 'event_time')
    #sliced_df = grouped_df.select('cc_num', 'num_trans_last_1d', 'avg_amt_last_1d')
    return sliced_df


def transform_row(sliced_df: DataFrame) -> list:
    logger.info('[Transform Spark DataFrame Row to SageMaker Feature Store Record]')
    records = []
    for row in sliced_df.rdd.collect():
        record = []
        # added tid -pmh
        tid, cc_num, num_trans_last_1d, avg_amt_last_1d, event_time = row
        if cc_num:
            # Added 'tid' -pmh
            record.append({'ValueAsString': str(tid), 'FeatureName': 'tid'})
            record.append({'ValueAsString': str(cc_num), 'FeatureName': 'cc_num'})
            record.append({'ValueAsString': str(num_trans_last_1d), 'FeatureName': 'num_trans_last_1d'})
            record.append({'ValueAsString': str(round(avg_amt_last_1d, 2)), 'FeatureName': 'avg_amt_last_1d'})
            # Added 'event_time' -pmh
            record.append({'ValueAsString': str(event_time), 'FeatureName': 'event_time'})
            records.append(record)
    return records


def write_to_feature_store(records: list) -> None:
    logger.info('[Write Grouped Features to SageMaker Online Feature Store]')
    success, fail = 0, 0
    for record in records:
        event_time_feature = {
                'FeatureName': 'trans_time',
                'ValueAsString': str(int(round(time.time())))
            }
        record.append(event_time_feature)
        response = feature_store_client.put_record(FeatureGroupName=FEATURE_GROUP, Record=record)
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            success += 1
        else:
            fail += 1
    logger.info('Success = {}'.format(success))
    logger.info('Fail = {}'.format(fail))
    ##assert success == TOTAL_UNIQUE_USERS
    assert fail == 0


def run_spark_job():
    spark = SparkSession.builder.appName('PySparkJob').getOrCreate()
    args = parse_args()
    schema = define_schema()
    aggregated_features = aggregate_features(args, schema, spark)
    write_to_s3(args, aggregated_features)
    sliced_df = group_by_card_number(aggregated_features)
    records = transform_row(sliced_df)
    write_to_feature_store(records)
    
    
if __name__ == '__main__':
    run_spark_job()

Overwriting batch_aggregation.py


### Run SageMaker Processing Job

In [17]:
spark_processor = PySparkProcessor(base_job_name='sagemaker-processing', 
                                   framework_version='2.4', # spark version
                                   role=role, 
                                   instance_count=1, 
                                   instance_type='ml.r5.4xlarge', 
                                   env={'AWS_DEFAULT_REGION': boto3.Session().region_name},
                                   max_runtime_in_seconds=1200)

In [18]:
%%time

spark_processor.run(submit_app='batch_aggregation.py', 
                    arguments=['--s3_input_bucket', BUCKET, 
                               '--s3_input_key_prefix', INPUT_KEY_PREFIX, 
                               '--s3_output_bucket', BUCKET, 
                               '--s3_output_key_prefix', OUTPUT_KEY_PREFIX],
                    spark_event_logs_s3_uri='s3://{}/logs'.format(BUCKET),
                    logs=True)

Creating processing-job with name sagemaker-processing-2021-04-22-18-06-16-608



Job Name:  sagemaker-processing-2021-04-22-18-06-16-608
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-572539092864/sagemaker-processing-2021-04-22-18-06-16-608/input/code/batch_aggregation.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-572539092864/logs', 'LocalPath': '/opt/ml/processing/spark-events/', 'S3UploadMode': 'Continuous'}}]
.............................04-22 18:10 smspark.cli  INFO     Parsing arguments. argv: ['/usr/local/bin/smspark-submit', '--local-spark-event-logs-dir', '/opt/ml/processing/spark-events/', '/opt/ml/processing/input/code/batch_aggregation.py', '--s3_input_bucket', 'sagemaker-us-east-1-572539092864', '--s3_input_key_prefix', 'sagemaker-featurestore-demo/raw', '--s3_outpu

21/04/22 18:11:09 INFO yarn.Client: Uploading resource file:/tmp/spark-f4b0ee0b-db78-4379-a6b1-bf65e03b66f5/__spark_libs__6926274954038324707.zip -> hdfs://10.2.66.31/user/root/.sparkStaging/application_1619115061235_0001/__spark_libs__6926274954038324707.zip
21/04/22 18:11:09 INFO hdfs.StateChange: BLOCK* allocate blk_1073741825_1001, replicas=10.2.66.31:50010 for /user/root/.sparkStaging/application_1619115061235_0001/__spark_libs__6926274954038324707.zip
21/04/22 18:11:11 INFO datanode.DataNode: Receiving BP-1441166884-10.2.66.31-1619115058139:blk_1073741825_1001 src: /10.2.66.31:49642 dest: /10.2.66.31:50010
21/04/22 18:11:11 INFO DataNode.clienttrace: src: /10.2.66.31:49642, dest: /10.2.66.31:50010, bytes: 134217728, op: HDFS_WRITE, cliID: DFSClient_NONMAPREDUCE_2057472956_27, offset: 0, srvID: 5eadf237-69d2-4446-87f7-3c63d05abb4c, blockid: BP-1441166884-10.2.66.31-1619115058139:blk_1073741825_1001, duration(ns): 185200597
21/04/22 18:11:11 INFO datanode.DataNode: PacketResponder:

21/04/22 18:11:14 INFO yarn.Client: Submitting application application_1619115061235_0001 to ResourceManager
21/04/22 18:11:14 INFO capacity.CapacityScheduler: Application 'application_1619115061235_0001' is submitted without priority hence considering default queue/cluster priority: 0
21/04/22 18:11:14 INFO capacity.CapacityScheduler: Priority '0' is acceptable in queue : default for application: application_1619115061235_0001
21/04/22 18:11:14 WARN rmapp.RMAppImpl: The specific max attempts: 0 for application: 1 is invalid, because it is out of the range [1, 1]. Use the global max attempts instead.
21/04/22 18:11:14 INFO resourcemanager.ClientRMService: Application with id 1 submitted by user root
21/04/22 18:11:14 INFO rmapp.RMAppImpl: Storing application with id application_1619115061235_0001
21/04/22 18:11:14 INFO resourcemanager.RMAuditLogger: USER=root#011IP=10.2.66.31#011OPERATION=Submit Application Request#011TARGET=ClientRMService#011RESULT=SUCCESS#011APPID=application_161911

21/04/22 18:11:23 INFO monitor.ContainersMonitorImpl: container_1619115061235_0001_01_000002's ip = 10.2.66.31, and hostname = algo-1
21/04/22 18:11:23 INFO monitor.ContainersMonitorImpl: Skipping monitoring container container_1619115061235_0001_01_000002 since CPU usage is not yet available.
21/04/22 18:11:24 INFO cluster.YarnSchedulerBackend$YarnDriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (10.2.66.31:43634) with ID 1
[Write Aggregated Features to S3]
21/04/22 18:11:24 INFO storage.BlockManagerMasterEndpoint: Registering block manager algo-1:45775 with 59.0 GB RAM, BlockManagerId(1, algo-1, 45775, None)
21/04/22 18:11:24 INFO scheduler.AppSchedulingInfo: checking for deactivate of application :application_1619115061235_0001
21/04/22 18:11:24 INFO datasources.FileSourceStrategy: Pruning directories with: 
21/04/22 18:11:24 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
21/04/22 18:11:24 INFO datasources.FileSourceStrategy: Output Data Sch

[/var/log/yarn/userlogs/application_1619115061235_0001/container_1619115061235_0001_01_000002/stderr] 21/04/22 18:11:32 INFO storage.ShuffleBlockFetcherIterator: Started 0 remote fetches in 0 ms
[/var/log/yarn/userlogs/application_1619115061235_0001/container_1619115061235_0001_01_000002/stderr] 21/04/22 18:11:32 INFO storage.ShuffleBlockFetcherIterator: Getting 10 non-empty blocks including 10 local blocks and 0 remote blocks
[/var/log/yarn/userlogs/application_1619115061235_0001/container_1619115061235_0001_01_000002/stderr] 21/04/22 18:11:32 INFO storage.ShuffleBlockFetcherIterator: Started 0 remote fetches in 1 ms
[/var/log/yarn/userlogs/application_1619115061235_0001/container_1619115061235_0001_01_000002/stderr] 21/04/22 18:11:33 INFO storage.ShuffleBlockFetcherIterator: Getting 10 non-empty blocks including 10 local blocks and 0 remote blocks
[/var/log/yarn/userlogs/application_1619115061235_0001/container_1619115061235_0001_01_000002/stderr] 21/04/22 18:11:33 INFO storage.Shuff

04-22 18:12 sagemaker-spark-event-logs-publisher INFO     Got spark event logs file: application_1619115061235_0001.inprogress
04-22 18:12 root         INFO     copying /tmp/spark-events/application_1619115061235_0001.inprogress to /opt/ml/processing/spark-events/application_1619115061235_0001
04-22 18:12 sagemaker-spark-event-logs-publisher INFO     Got spark event logs file: application_1619115061235_0001.inprogress
04-22 18:12 root         INFO     copying /tmp/spark-events/application_1619115061235_0001.inprogress to /opt/ml/processing/spark-events/application_1619115061235_0001
04-22 18:12 sagemaker-spark-event-logs-publisher INFO     Got spark event logs file: application_1619115061235_0001.inprogress
04-22 18:12 root         INFO     copying /tmp/spark-events/application_1619115061235_0001.inprogress to /opt/ml/processing/spark-events/application_1619115061235_0001
04-22 18:13 sagemaker-spark-event-logs-publisher INFO     Got spark event logs file: application_1619115061235_0001

04-22 18:14 sagemaker-spark-event-logs-publisher INFO     Got spark event logs file: application_1619115061235_0001
04-22 18:14 root         INFO     copying /tmp/spark-events/application_1619115061235_0001 to /opt/ml/processing/spark-events/application_1619115061235_0001

CPU times: user 954 ms, sys: 80.3 ms, total: 1.03 s
Wall time: 8min 16s
